In [1]:
! pip install pymoo ortools

  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached Deprecated-1.2.18-py2.py3-none-any.whl.metadata (5.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 945.4/945.4 kB 940.1 kB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 530.5 kB/s  0:00:59 eta 0:00:02
Using cached Deprecated-1.2.18-py2.py3-none-any.whl (10.0 kB)
Using cached dill-0.4.0-py3-none-any.whl (119 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [pymoo]m 9/10 [pymoo]


In [11]:
import numpy as np
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.problem import Problem
from pymoo.termination import get_termination
from pymoo.optimize import minimize
from ortools.sat.python import cp_model

# CP function: given job sequence, compute makespan and total tardiness
def evaluate_schedule(order):
    model = cp_model.CpModel()
    
    # Data
    M = 2  # machines
    J = 3  # jobs
    processing_times = [
        [3, 2],  # Job 0
        [2, 1],  # Job 1
        [4, 3],  # Job 2
    ]
    due_dates = [5, 6, 7]
    
    # Convert order to integers and validate
    order = [int(round(x)) for x in order]
    
    # Ensure valid job indices (0, 1, 2)
    order = [max(0, min(J-1, x)) for x in order]
    
    # Start time variables
    start = {}
    end = {}
    for j in range(J):
        for m in range(M):
            start[(j, m)] = model.NewIntVar(0, 50, f"start_{j}_{m}")
            end[(j, m)]   = model.NewIntVar(0, 50, f"end_{j}_{m}")
            model.Add(end[(j, m)] == start[(j, m)] + processing_times[j][m])
    
    # Precedence within jobs (job must finish on machine m before starting on m+1)
    for j in range(J):
        for m in range(M-1):
            model.Add(end[(j, m)] <= start[(j, m+1)])
    
    # No-overlap per machine according to order
    for m in range(M):
        # Get unique jobs for this machine (in case of duplicates in order)
        seen = set()
        unique_order = []
        for j in order:
            if j not in seen:
                seen.add(j)
                unique_order.append(j)
        
        # Add precedence constraints for the unique order
        for i in range(len(unique_order)-1):
            j1 = unique_order[i]
            j2 = unique_order[i+1]
            model.Add(start[(j2, m)] >= end[(j1, m)])
    
    # Makespan variable
    makespan = model.NewIntVar(0, 100, "makespan")
    model.AddMaxEquality(makespan, [end[(j, M-1)] for j in range(J)])
    
    # Total tardiness
    tardiness_vars = []
    for j in range(J):
        t_var = model.NewIntVar(0, 100, f"tard_{j}")
        model.Add(t_var >= end[(j, M-1)] - due_dates[j])
        tardiness_vars.append(t_var)
    total_tardiness = model.NewIntVar(0, 100, "total_tard")
    model.Add(total_tardiness == sum(tardiness_vars))
    
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 1
    status = solver.Solve(model)
    
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        return solver.Value(makespan), solver.Value(total_tardiness)
    else:
        return 9999, 9999  # infeasible/penalized

# Define the pymoo Problem class
class JSSP_CP_Problem(Problem):
    def __init__(self):
        super().__init__(
            n_var=3,           # 3 jobs to order
            n_obj=2,           # 2 objectives (makespan, tardiness)
            n_constr=0,        # no constraints
            xl=0,              # lower bound
            xu=2.999,          # upper bound (just below 3)
            type_var=float     # use float internally, convert in evaluate
        )
    
    def _evaluate(self, X, out, *args, **kwargs):
        F = []
        for row in X:
            # Convert continuous values to job indices
            order = [int(round(x)) for x in row]
            # Ensure valid range [0, 2]
            order = [max(0, min(2, x)) for x in order]
            
            ms, tt = evaluate_schedule(order)
            F.append([ms, tt])
        out["F"] = np.array(F)

# Run NSGA-II with the standard problem
print("Running standard NSGA-II:")
problem = JSSP_CP_Problem()
algorithm = NSGA2(pop_size=10)
termination = get_termination("n_gen", 5)

res = minimize(problem, algorithm, termination, verbose=True)

print("\nPareto front solutions (order, makespan, tardiness):")
if res.X is not None:
    for x, f in zip(res.X, res.F):
        order = [int(round(val)) for val in x]
        print(f"Order: {order}, Makespan: {f[0]:.1f}, Tardiness: {f[1]:.1f}")
else:
    print("No solutions found")

# Alternative approach with integer handling
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.repair.rounding import RoundingRepair

class JSSP_Integer_Problem(Problem):
    def __init__(self):
        super().__init__(
            n_var=3,
            n_obj=2,
            n_constr=0,
            xl=np.array([0, 0, 0]),
            xu=np.array([2, 2, 2]),
            type_var=int
        )
    
    def _evaluate(self, X, out, *args, **kwargs):
        F = []
        for row in X:
            # Ensure integers
            order = [int(x) for x in row]
            ms, tt = evaluate_schedule(order)
            F.append([ms, tt])
        out["F"] = np.array(F)

# Create algorithm with integer-specific operators
algorithm_int = NSGA2(
    pop_size=20,
    sampling=IntegerRandomSampling(),
    crossover=SBX(prob=0.9, eta=15, vtype=float, repair=RoundingRepair()),
    mutation=PM(prob=1.0, eta=20, vtype=float, repair=RoundingRepair()),
    eliminate_duplicates=True
)

print("\n\nRunning with integer-specific operators:")
problem2 = JSSP_Integer_Problem()
termination2 = get_termination("n_gen", 10)
res2 = minimize(problem2, algorithm_int, termination2, verbose=True)

if res2.X is not None:
    print("\nResults with integer operators:")
    for x, f in zip(res2.X, res2.F):
        print(f"Order: {x}, Makespan: {f[0]:.1f}, Tardiness: {f[1]:.1f}")

# Additional approach: Custom permutation representation
print("\n\nAlternative: Using permutation representation")

class JSSP_Permutation(Problem):
    def __init__(self):
        super().__init__(
            n_var=3,
            n_obj=2,
            n_constr=0,
            xl=0,
            xu=2,
            type_var=int
        )
    
    def _evaluate(self, X, out, *args, **kwargs):
        F = []
        for row in X:
            # Create a permutation from the values
            # Sort indices by values to create a permutation
            indices = np.argsort(row)
            # Use indices as job order
            ms, tt = evaluate_schedule(indices)
            F.append([ms, tt])
        out["F"] = np.array(F)

problem3 = JSSP_Permutation()
algorithm3 = NSGA2(pop_size=15)
res3 = minimize(problem3, algorithm3, termination, verbose=True)

if res3.X is not None:
    print("\nResults with permutation encoding:")
    for x, f in zip(res3.X, res3.F):
        order = np.argsort(x)
        print(f"Order: {order}, Makespan: {f[0]:.1f}, Tardiness: {f[1]:.1f}")

Running standard NSGA-II:
n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       10 |      3 |             - |             -
     2 |       20 |      6 |  0.000000E+00 |             f
     3 |       30 |     10 |  0.000000E+00 |             f
     4 |       40 |     10 |  0.000000E+00 |             f
     5 |       50 |     10 |  0.000000E+00 |             f

Pareto front solutions (order, makespan, tardiness):
Order: [3, 2, 2], Makespan: 7.0, Tardiness: 0.0
Order: [2, 2, 2], Makespan: 7.0, Tardiness: 0.0
Order: [3, 2, 2], Makespan: 7.0, Tardiness: 0.0
Order: [3, 2, 2], Makespan: 7.0, Tardiness: 0.0
Order: [3, 2, 2], Makespan: 7.0, Tardiness: 0.0
Order: [2, 2, 2], Makespan: 7.0, Tardiness: 0.0
Order: [2, 3, 2], Makespan: 7.0, Tardiness: 0.0
Order: [3, 2, 2], Makespan: 7.0, Tardiness: 0.0
Order: [2, 3, 2], Makespan: 7.0, Tardiness: 0.0
Order: [3, 2, 2], Makespan: 7.0, Tardiness: 0.0


Running with integer-specific operators:
n_gen  |  n_eval  | n_nds  |      eps      